In [1]:
import pip
#import CONFIG as CONFIG
from bs4 import BeautifulSoup
import pandas as pd
import pathlib
import os
import ast
import sys
import requests
import numpy as np
import sklearn
import time
#from html_table_parser.parser import HTMLTableParser
import urllib.request
from random import randint

#MAX_CRAWL_SLEEP_TIME = CONFIG["crawler"]["MAX_CRAWL_SLEEP_TIME"]


In [2]:
#western developed
EU_Arr=["france","uk","italy","spain",
        "portugal","poland","austria",
        "denmark","finland","greece",
        "hungary","ireland",
        "netherlands","romania",
        "sweden","iceland","belgium","norway","Switzerland"]

westernC_Arr=["japan","australia","canada","south-korea","new-zealand",
              "usa","israel"]#8

arabArr=[ 'jordan', 'lebanon', 'mauritania',
          'morocco',
           'syria', 'tunisia',
          'algeria', 'egypt',
          'iraq','turkey','iran']#11

superNationArr=["china","india"]#3


AfricaC_Arr=["nigeria","ethiopia","democratic-republic-congo","tanzania","south-africa",
             "kenya","uganda","algeria","angola","mozambique",
             "ghana","madagascar","cameroon",
             "mali","malawi","zambia",
             "zimbabwe","guinea","rwanda",
             "burundi","togo",
             "congo","central-african-republic","mauritania",
             "gabon",
             "equatorial-guinea","mauritius" ]#27


other_asian_arr=["nepal","Thailand","Malaysia","Philippines","Afghanistan","Pakistan"]#6

AmericaC_Arr=["brazil","argentina","peru","chile","colombia"]#5

countriesArr=[]
countriesArr.extend(EU_Arr)
countriesArr.extend(westernC_Arr)
countriesArr.extend(arabArr)
countriesArr.extend(superNationArr)
countriesArr.extend(AfricaC_Arr)
countriesArr.extend(other_asian_arr)
countriesArr.extend(AmericaC_Arr)
len(countriesArr)

77

In [3]:
Dict = {'countries':[],'Year':[], 'CO2 Total Mt':[],'CO2 Kg/1k$':[],'CO2 Tons per capita':[],
          'Generation GWh':[],'consumption GWh':[],'consumption per capita kW':[],'installed capacity MW':[],'Renewable installed capacity MW':[],
          'Renewable generation GWh':[],'Renewable percentage %':[],'Population':[]}

num_rows =42
gap_to_fill_nan=0
#loop=0
for i in countriesArr:
    print(i)
    time.sleep(randint(0, 2))
    Co2_URL = "https://countryeconomy.com/energy-and-environment/co2-emissions"
    html = requests.get(Co2_URL+'/'+i).text
    Soup = BeautifulSoup(html, "html.parser")
    Right_table=Soup.find('table',id = "tb0")
    j = 0
    for row in Right_table.findAll('tr'):
        if j== num_rows:
            break
        j=j+1
        cells=row.findAll('td')
        if len(cells)==4:
            Dict['countries'].append(i)
            Dict['Year'].append(cells[0].find(text=True))
            Dict['CO2 Total Mt'].append(cells[1].find(text=True))
            Dict['CO2 Kg/1k$'].append(cells[2].find(text=True))
            Dict['CO2 Tons per capita'].append(cells[3].find(text=True))

    j=0
    gap_to_fill_nan=0
    Electricity_consumption = "https://countryeconomy.com/energy-and-environment/electricity-consumption"
    html = requests.get(Electricity_consumption+'/'+i).text
    Soup = BeautifulSoup(html, "html.parser")
    Right_table=Soup.find('table',id = "tb0")
# sometimes the table starts from 2019 instead from 2020 so take that shit:
    if Right_table.find('td').find(text=True)!='2020':
        Dict['consumption GWh'].append(np.nan)
        Dict['consumption per capita kW'].append(np.nan)
        gap_to_fill_nan=gap_to_fill_nan+1

    for row in Right_table.findAll('tr'):
        if j== num_rows:
            break
        j=j+1
        cells=row.findAll('td')
        if len(cells)==4:
            Dict['consumption GWh'].append(cells[2].find(text=True))
            Dict['consumption per capita kW'].append(cells[3].find(text=True))
        gap_to_fill_nan=gap_to_fill_nan+1
    for ro in range(0,num_rows-gap_to_fill_nan):
        Dict['consumption GWh'].append(np.nan)
        Dict['consumption per capita kW'].append(np.nan)
    gap_to_fill_nan=0
    j=0
    Electricity_generation = "https://countryeconomy.com/energy-and-environment/electricity-generation"
    html = requests.get(Electricity_generation+'/'+i).text
    Soup = BeautifulSoup(html, "html.parser")
    Right_table=Soup.find('table',id = "tb0")

    if Right_table.find('td').find(text=True)!='2020':
        Dict['installed capacity MW'].append(np.nan)
        Dict['Generation GWh'].append(np.nan)
        Dict['Renewable installed capacity MW'].append(np.nan)
        Dict['Renewable generation GWh'].append(np.nan)
        Dict['Renewable percentage %'].append(np.nan)
        gap_to_fill_nan=gap_to_fill_nan+1

    for row in Right_table.findAll('tr') :
        if j== num_rows:
            break
        j=j+1
        cells=row.findAll('td')
        if len(cells)==6:
            Dict['installed capacity MW'].append(cells[1].find(text=True))
            Dict['Generation GWh'].append(cells[2].find(text=True))
            Dict['Renewable installed capacity MW'].append(cells[3].find(text=True))
            Dict['Renewable generation GWh'].append(cells[4].find(text=True))
            Dict['Renewable percentage %'].append(cells[5].find(text=True))
            gap_to_fill_nan=gap_to_fill_nan+1
    for ro in range(1,num_rows-gap_to_fill_nan):
        Dict['installed capacity MW'].append(np.nan)
        Dict['Generation GWh'].append(np.nan)
        Dict['Renewable installed capacity MW'].append(np.nan)
        Dict['Renewable generation GWh'].append(np.nan)
        Dict['Renewable percentage %'].append(np.nan)


    j=0
    Population = "https://countryeconomy.com/demography/population"
    html = requests.get(Population+'/'+i).text
    Soup = BeautifulSoup(html, "html.parser")
    Right_table=Soup.find('table',id = "tb0")
    for row in Right_table.findAll('tr') :
        if j== num_rows:
            break
        j=j+1
        cells=row.findAll('td')
        if len(cells)==5:
            Dict['Population'].append(cells[4].find(text=True))


df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in Dict.items()]))
df.to_csv('Big Table.csv')
df.info()

france
uk
italy
spain


KeyboardInterrupt: 